<a href="https://colab.research.google.com/github/saadoonhammad/avamet-data/blob/main/Avamet_Interpolation_Linear_Xgboost.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install -U kaleido

In [ ]:
import os
import random
from os import listdir
from io import StringIO
from os.path import join
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import plotly.express as px
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import plotly.express as px
from plotly.io import write_image
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

# Values in missing interval

Before data imputation, it is important to calculate the length of consecutive missing values in each dataset.

In [ ]:
missing_all= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/Avamet/missing_values_all_years.csv')
missing_all

,Unnamed: 0,file_name_21,missing_values_21,file_name_22,missing_values_22,file_name_23,missing_values_23
0,0,c01m045e01_2021.csv,6,c01m045e01_2022.csv,6,c01m045e01_2023.csv,7
1,1,c01m061e01_2021.csv,228,c01m061e01_2022.csv,875,c01m061e01_2023.csv,8
2,2,c01m080e01_2021.csv,6,c01m080e01_2022.csv,246,c01m080e01_2023.csv,97
3,3,c01m083e01_2021.csv,12,c01m083e01_2022.csv,23,c01m083e01_2023.csv,129
4,4,c01m127e01_2021.csv,406,c01m127e01_2022.csv,204,c01m127e01_2023.csv,11
5,5,c01m141e01_2021.csv,241,c01m141e01_2022.csv,7,c01m141e01_2023.csv,10
6,6,c02m026e02_2021.csv,17,c02m026e02_2022.csv,1419,c02m026e02_2023.csv,240
7,7,c02m042e02_2021.csv,483,c02m042e02_2022.csv,215,c02m042e02_2023.csv,95
8,8,c02m042e03_2021.csv,1147,c02m042e03_2022.csv,6671,c02m042e03_2023.csv,1504
9,9,c02m051e03_2021.csv,210,c02m051e03_2022.csv,8,c02m051e03_2023.csv,11


In [ ]:
import os
import pandas as pd


def process_file(file_path, file_name):
    df = pd.read_csv(file_path)

    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df['is_missing'] = df['temp_value'].isnull()


    df['missing_group'] = (df['is_missing'] != df['is_missing'].shift()).cumsum()
    missing_data_df = df[df['is_missing']]

    missing_counts = missing_data_df.groupby('missing_group')['temp_value'].size().reset_index(name='missing_count')
    missing_counts['file_name'] = file_name

    return missing_counts

# Function to process all files in a directory and save results in one file
def process_all_files_in_folder(folder_path, output_file):
    file_list = [f for f in os.listdir(folder_path) if f.endswith('.csv')]

    all_missing_counts = []

    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)
        missing_counts = process_file(file_path, file_name)
        all_missing_counts.append(missing_counts)

    # Concatenate all the missing counts into a single DataFrame
    combined_missing_counts = pd.concat(all_missing_counts, ignore_index=True)

    # Save the combined missing counts to a single CSV file
    combined_missing_counts.to_csv(output_file, index=False)
    print(f"All missing counts saved to {output_file}")

folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Resampled/"
output_file = 'combined_missing_data_counts_23.csv'

process_all_files_in_folder(folder_path, output_file)


All missing counts saved to combined_missing_data_counts_23.csv


# Linear Interpolation

**150 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'  # Default year


        data = pd.read_csv(file_path)

        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]

        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        gap_size = 150
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='linear')

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f} | MAE: {mae:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_results_file = os.path.join(output_folder_path, 'comparison_linear_interpolation_150.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nLinear interpolation completed for all files. Results saved to {comparison_results_file}.")


Processed file: c01m045e01_2021.csv | RMSE: 5.5961 | MAPE: 0.4204 | MAE: 4.6936
Processed file: c05m124e01_2021.csv | RMSE: 2.9491 | MAPE: 0.1587 | MAE: 2.2611
Processed file: c05m105e08_2022.csv | RMSE: 2.5564 | MAPE: 0.0970 | MAE: 2.1402
Processed file: c05m105e08_2023.csv | RMSE: 4.4090 | MAPE: 0.3090 | MAE: 3.4873
Processed file: c05m124e01_2023.csv | RMSE: 5.9770 | MAPE: 0.3474 | MAE: 4.8061
Processed file: c01m045e01_2022.csv | RMSE: 6.2668 | MAPE: 0.2828 | MAE: 4.8389
Processed file: c01m045e01_2023.csv | RMSE: 5.2205 | MAPE: 0.5099 | MAE: 4.5403
Processed file: c05m124e01_2022.csv | RMSE: 4.4527 | MAPE: 0.1723 | MAE: 3.5843
Processed file: c05m105e08_2021.csv | RMSE: 1.9439 | MAPE: 0.1234 | MAE: 1.6323

Linear interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_linear_interpolation_150.csv.


**300 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)

comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 300
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='linear')

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_results_file = os.path.join(output_folder_path, 'comparison_linear_interpolation_300.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nLinear interpolation completed for all files. Results saved to {comparison_results_file}.")


Processed file: c01m045e01_2021.csv | RMSE: 4.8333 | MAPE: 0.3375
Processed file: c05m124e01_2021.csv | RMSE: 2.1121 | MAPE: 0.1057
Processed file: c05m105e08_2022.csv | RMSE: 3.4019 | MAPE: 0.1342
Processed file: c05m105e08_2023.csv | RMSE: 2.8927 | MAPE: 0.2121
Processed file: c05m124e01_2023.csv | RMSE: 4.1008 | MAPE: 0.2287
Processed file: c01m045e01_2022.csv | RMSE: 5.1312 | MAPE: 0.2195
Processed file: c01m045e01_2023.csv | RMSE: 4.1902 | MAPE: 0.3929
Processed file: c05m124e01_2022.csv | RMSE: 5.2449 | MAPE: 0.2054
Processed file: c05m105e08_2021.csv | RMSE: 2.2472 | MAPE: 0.1085

Linear interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_linear_interpolation_300.csv.


**450 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'


        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 450
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='linear')

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_results_file = os.path.join(output_folder_path, 'comparison_linear_interpolation_450.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nLinear interpolation completed for all files. Results saved to {comparison_results_file}.")


Processed file: c01m045e01_2021.csv | RMSE: 6.7331 | MAPE: 0.5188
Processed file: c05m124e01_2021.csv | RMSE: 3.2259 | MAPE: 0.1610
Processed file: c05m105e08_2022.csv | RMSE: 4.8845 | MAPE: 0.2062
Processed file: c05m105e08_2023.csv | RMSE: 4.6570 | MAPE: 0.3679
Processed file: c05m124e01_2023.csv | RMSE: 4.0127 | MAPE: 0.2246
Processed file: c01m045e01_2022.csv | RMSE: 5.6198 | MAPE: 0.2444
Processed file: c01m045e01_2023.csv | RMSE: 4.8964 | MAPE: 0.4645
Processed file: c05m124e01_2022.csv | RMSE: 6.1145 | MAPE: 0.2496
Processed file: c05m105e08_2021.csv | RMSE: 2.5844 | MAPE: 0.1372

Linear interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_linear_interpolation_450.csv.


**600 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 600
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='linear')

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_results_file = os.path.join(output_folder_path, 'comparison_linear_interpolation_600.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nLinear interpolation completed for all files. Results saved to {comparison_results_file}.")


Processed file: c01m045e01_2021.csv | RMSE: 5.4513 | MAPE: 0.3849
Processed file: c05m124e01_2021.csv | RMSE: 3.6705 | MAPE: 0.1834
Processed file: c05m105e08_2022.csv | RMSE: 5.3082 | MAPE: 0.2240
Processed file: c05m105e08_2023.csv | RMSE: 5.3007 | MAPE: 0.4089
Processed file: c05m124e01_2023.csv | RMSE: 6.0832 | MAPE: 0.3675
Processed file: c01m045e01_2022.csv | RMSE: 7.0436 | MAPE: 0.3137
Processed file: c01m045e01_2023.csv | RMSE: 4.2661 | MAPE: 0.3972
Processed file: c05m124e01_2022.csv | RMSE: 7.1581 | MAPE: 0.2996
Processed file: c05m105e08_2021.csv | RMSE: 2.9741 | MAPE: 0.1586

Linear interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_linear_interpolation_600.csv.


# Spline Interpolation

**150 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error


input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 150
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='spline', order=3)

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")


comparison_df = pd.DataFrame(comparison_results)
comparison_results_file = os.path.join(output_folder_path, 'comparison_spline_interpolation_150.csv')
comparison_df.to_csv(comparison_results_file, index=False)
print(f"\nSpline interpolation completed for all files. Results saved to {comparison_results_file}.")

Processed file: c01m045e01_2021.csv | RMSE: 8.1125 | MAPE: 0.6183
Processed file: c05m124e01_2021.csv | RMSE: 3.5427 | MAPE: 0.2014
Processed file: c05m105e08_2022.csv | RMSE: 4.5828 | MAPE: 0.1837
Processed file: c05m105e08_2023.csv | RMSE: 6.1884 | MAPE: 0.4462
Processed file: c05m124e01_2023.csv | RMSE: 10.3698 | MAPE: 0.6265
Processed file: c01m045e01_2022.csv | RMSE: 11.3412 | MAPE: 0.5373
Processed file: c01m045e01_2023.csv | RMSE: 8.5621 | MAPE: 0.8395
Processed file: c05m124e01_2022.csv | RMSE: 8.4695 | MAPE: 0.3397
Processed file: c05m105e08_2021.csv | RMSE: 2.7534 | MAPE: 0.1787

Spline interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_spline_interpolation_150.csv.


**300 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error


input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 300
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='spline', order=3)

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")


comparison_df = pd.DataFrame(comparison_results)
comparison_df['File Name'] = pd.Categorical(comparison_df['File Name'], categories=[
    'c01m045e01_2021', 'c05m124e01_2021', 'c05m105e08_2021',
    'c01m045e01_2022', 'c05m124e01_2022', 'c05m105e08_2022',
    'c01m045e01_2023', 'c05m124e01_2023', 'c05m105e08_2023'
], ordered=True)
comparison_df.sort_values(by=['File Name'], inplace=True)
comparison_results_file = os.path.join(output_folder_path, 'comparison_spline_interpolation_300.csv')
comparison_df.to_csv(comparison_results_file, index=False)
print(f"\nSpline interpolation completed for all files. Results saved to {comparison_results_file}.")

Processed file: c01m045e01_2021.csv | RMSE: 8.0640 | MAPE: 0.5921
Processed file: c05m124e01_2021.csv | RMSE: 3.5937 | MAPE: 0.1880
Processed file: c05m105e08_2022.csv | RMSE: 9.7346 | MAPE: 0.4136
Processed file: c05m105e08_2023.csv | RMSE: 7.3297 | MAPE: 0.5714
Processed file: c05m124e01_2023.csv | RMSE: 5.2852 | MAPE: 0.3031
Processed file: c01m045e01_2022.csv | RMSE: 13.9212 | MAPE: 0.6444
Processed file: c01m045e01_2023.csv | RMSE: 10.1595 | MAPE: 0.9463
Processed file: c05m124e01_2022.csv | RMSE: 10.6673 | MAPE: 0.4538
Processed file: c05m105e08_2021.csv | RMSE: 3.4289 | MAPE: 0.1942

Spline interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_spline_interpolation_300.csv.


**450 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 450
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='spline', order=3)

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_df['File Name'] = pd.Categorical(comparison_df['File Name'], categories=[
    'c01m045e01_2021', 'c05m124e01_2021', 'c05m105e08_2021',
    'c01m045e01_2022', 'c05m124e01_2022', 'c05m105e08_2022',
    'c01m045e01_2023', 'c05m124e01_2023', 'c05m105e08_2023'
], ordered=True)
comparison_df.sort_values(by=['File Name'], inplace=True)
comparison_results_file = os.path.join(output_folder_path, 'comparison_spline_interpolation_450.csv')
comparison_df.to_csv(comparison_results_file, index=False)
print(f"\nSpline interpolation completed for all files. Results saved to {comparison_results_file}.")

Processed file: c01m045e01_2021.csv | RMSE: 15.7474 | MAPE: 1.2238
Processed file: c05m124e01_2021.csv | RMSE: 6.4812 | MAPE: 0.3469
Processed file: c05m105e08_2022.csv | RMSE: 14.1173 | MAPE: 0.6166
Processed file: c05m105e08_2023.csv | RMSE: 16.7273 | MAPE: 1.3312
Processed file: c05m124e01_2023.csv | RMSE: 19.7677 | MAPE: 1.2083
Processed file: c01m045e01_2022.csv | RMSE: 29.6028 | MAPE: 1.3555
Processed file: c01m045e01_2023.csv | RMSE: 19.3974 | MAPE: 1.8240
Processed file: c05m124e01_2022.csv | RMSE: 23.9803 | MAPE: 1.0511
Processed file: c05m105e08_2021.csv | RMSE: 5.4019 | MAPE: 0.2960

Spline interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_spline_interpolation_450.csv.


**600 GAP SIZE**

In [ ]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)

        start_gap = 6700
        gap_size = 600
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan
        data_filtered_with_nan['temp_value_imp'] = data_filtered_with_nan['temp_value_imp'].interpolate(method='spline', order=3)

        original_values = data_filtered.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']
        imputed_values = data_filtered_with_nan.loc[start_gap:start_gap + gap_size - 1, 'temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae=mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'Year': year, 'File Name': file_name.split('.')[0], 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})

        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

comparison_df = pd.DataFrame(comparison_results)
comparison_df['File Name'] = pd.Categorical(comparison_df['File Name'], categories=[
    'c01m045e01_2021', 'c05m124e01_2021', 'c05m105e08_2021',
    'c01m045e01_2022', 'c05m124e01_2022', 'c05m105e08_2022',
    'c01m045e01_2023', 'c05m124e01_2023', 'c05m105e08_2023'
], ordered=True)
comparison_df.sort_values(by=['File Name'], inplace=True)
comparison_results_file = os.path.join(output_folder_path, 'comparison_spline_interpolation_600.csv')
comparison_df.to_csv(comparison_results_file, index=False)
print(f"\nSpline interpolation completed for all files. Results saved to {comparison_results_file}.")

Processed file: c01m045e01_2021.csv | RMSE: 13.6892 | MAPE: 0.9322
Processed file: c05m124e01_2021.csv | RMSE: 6.2446 | MAPE: 0.3147
Processed file: c05m105e08_2022.csv | RMSE: 6.8165 | MAPE: 0.2907
Processed file: c05m105e08_2023.csv | RMSE: 19.6346 | MAPE: 1.5183
Processed file: c05m124e01_2023.csv | RMSE: 29.3494 | MAPE: 1.7803
Processed file: c01m045e01_2022.csv | RMSE: 39.6371 | MAPE: 1.8087
Processed file: c01m045e01_2023.csv | RMSE: 18.1318 | MAPE: 1.6380
Processed file: c05m124e01_2022.csv | RMSE: 35.5082 | MAPE: 1.5495
Processed file: c05m105e08_2021.csv | RMSE: 4.7657 | MAPE: 0.2767

Linear interpolation completed for all files. Results saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Results/comparison_spline_interpolation_600.csv.


# XGBoost Data Imputation - Limited Selected Dataset

**Gap with 150 Values**

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import plotly.express as px
from plotly.io import write_image


input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Plots'
# output_folder_path = '/content/'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)


        # Introduce a gap of 150 consecutive NaN values
        start_gap = 7200
        gap_size = 150
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan

        # Feature Engineering: Add time-based features
        data_filtered_with_nan['time_step'] = range(len(data_filtered_with_nan))
        data_filtered_with_nan['hour'] = data_filtered_with_nan['timestamp'].dt.hour
        data_filtered_with_nan['minute'] = data_filtered_with_nan['timestamp'].dt.minute
        data_filtered_with_nan['day_of_week'] = data_filtered_with_nan['timestamp'].dt.dayofweek
        data_filtered_with_nan['month'] = data_filtered_with_nan['timestamp'].dt.month

        # Separate the data into known and missing parts
        known_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].notna()]
        missing_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].isna()]

        # Define features and target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X_known = known_data[features]
        y_known = known_data['temp_value_imp']
        X_missing = missing_data[features]

        # Train XGBoost regressor
        xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        xgb_model.fit(X_known, y_known)

        # Predict missing values
        missing_data['temp_value_imp'] = xgb_model.predict(X_missing)

        # Combine the imputed data with the known data
        imputed_data = data_filtered_with_nan.copy()
        imputed_data.loc[missing_data.index, 'temp_value_imp'] = missing_data['temp_value_imp']
        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']

        # Plot original vs imputed data for visualization using Plotly Express
        fig = px.line(
            missing_data,
            x=missing_data.index,
            y='temp_value_imp',
            width=1000, height=800,
            title=f'Original vs Imputed Data - {file_name}',
            labels={'temp_value_imp': 'Temperature', 'index': 'Time'},
            color_discrete_sequence=['blue']
        )
        fig.add_scatter(x=original_values.index, y=original_values.values, mode='lines', name='Original', line=dict(color='blue', dash='solid'))
        fig.add_scatter(x=missing_data.index, y=missing_data['temp_value_imp'], mode='lines', name='Imputed', line=dict(color='red', dash='solid'))


        # Add the name for imputed data in the legend
        fig.update_traces(name='Imputed', selector=dict(mode='lines'))

        # Save the plot as a PNG file
        plot_file_name = f"{file_name.split('.')[0]}_{gap_size}.png"
        plot_file_path = os.path.join(output_folder_path, plot_file_name)
        fig.write_image(plot_file_path, scale=2)

        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']
        imputed_values = missing_data['temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae = mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'File Name': file_name, 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        # print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}, MAE: {mae:.4f}")

# Save comparison results to a CSV file
comparison_df = pd.DataFrame(comparison_results)
comparison_df["Year"] = comparison_df["File Name"].str.extract(r"_(\d{4})\.csv")
comparison_df["Suffix"] = comparison_df["File Name"].str.extract(r"e(\d{2})_")
comparison_df = comparison_df.sort_values(by=["Year", "Suffix"])
comparison_df = comparison_df.drop(columns=["Year", "Suffix"])
comparison_results_file = os.path.join(output_folder_path, 'imputation_comparison_results_150.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nImputation completed for all files. Results saved to {comparison_results_file}.")


**Gap with 300 Values**

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
import plotly.express as px
from plotly.io import write_image

# Specify folder paths and file patterns
input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Plots'
# output_folder_path = '/content/'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

# Process all relevant files
for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-04-01') & (data['timestamp'] <= f'{year}-06-30')].reset_index(drop=True)


        # Introduce a gap of 300 consecutive NaN values in 'temp_value_imp'
        start_gap = 3400
        gap_size = 300
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan

        # Feature Engineering: Add time-based features
        data_filtered_with_nan['time_step'] = range(len(data_filtered_with_nan))
        data_filtered_with_nan['hour'] = data_filtered_with_nan['timestamp'].dt.hour
        data_filtered_with_nan['minute'] = data_filtered_with_nan['timestamp'].dt.minute
        data_filtered_with_nan['day_of_week'] = data_filtered_with_nan['timestamp'].dt.dayofweek
        data_filtered_with_nan['month'] = data_filtered_with_nan['timestamp'].dt.month


        # Separate the data into known and missing parts
        known_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].notna()]
        missing_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].isna()]

        # Define features and target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X_known = known_data[features]
        y_known = known_data['temp_value_imp']
        X_missing = missing_data[features]

        # Train XGBoost regressor
        xgb_model = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
        xgb_model.fit(X_known, y_known)

        # Predict missing values
        missing_data['temp_value_imp'] = xgb_model.predict(X_missing)

        # Combine the imputed data with the known data
        imputed_data = data_filtered_with_nan.copy()
        imputed_data.loc[missing_data.index, 'temp_value_imp'] = missing_data['temp_value_imp']
        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']

        # Plot original vs imputed data for visualization using Plotly Express
        fig = px.line(
            missing_data,
            x=missing_data.index,
            y='temp_value_imp',
            width=1000, height=800,
            title=f'Original vs Imputed Data - {file_name}',
            labels={'temp_value_imp': 'Temperature', 'index': 'Time'},
            color_discrete_sequence=['blue']
        )
        fig.add_scatter(x=original_values.index, y=original_values.values, mode='lines', name='Original', line=dict(color='blue', dash='solid'))
        fig.add_scatter(x=missing_data.index, y=missing_data['temp_value_imp'], mode='lines', name='Imputed', line=dict(color='red', dash='solid'))


        # Add the name for imputed data in the legend
        fig.update_traces(name='Imputed', selector=dict(mode='lines'))

        # Save the plot as a PNG file
        plot_file_name = f"{file_name.split('.')[0]}_{gap_size}.png"
        plot_file_path = os.path.join(output_folder_path, plot_file_name)
        fig.write_image(plot_file_path, scale=2)


        # Evaluate the performance (RMSE, MAPE)
        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']
        imputed_values = missing_data['temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae = mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'File Name': file_name, 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        # print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f} | MAE: {mae:.4f}")

# Save comparison results to a CSV file
comparison_df = pd.DataFrame(comparison_results)
comparison_df["Year"] = comparison_df["File Name"].str.extract(r"_(\d{4})\.csv")
comparison_df["Suffix"] = comparison_df["File Name"].str.extract(r"e(\d{2})_")
comparison_df = comparison_df.sort_values(by=["Year", "Suffix"])
comparison_df = comparison_df.drop(columns=["Year", "Suffix"])
comparison_results_file = os.path.join(output_folder_path, 'imputation_comparison_results_300.csv')
comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nImputation completed for all files. Results saved to {comparison_results_file}.")


**Gap with 450 Values**

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error
import plotly.express as px
from plotly.io import write_image

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Plots'
# output_folder_path = '/content/'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-06-01') & (data['timestamp'] <= f'{year}-08-30')].reset_index(drop=True)

        # Introduce a gap of 150 consecutive NaN values in 'temp_value_imp'
        start_gap = 6700
        gap_size = 450
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan

        # Feature Engineering: Add time-based features
        data_filtered_with_nan['time_step'] = range(len(data_filtered_with_nan))
        data_filtered_with_nan['hour'] = data_filtered_with_nan['timestamp'].dt.hour
        data_filtered_with_nan['minute'] = data_filtered_with_nan['timestamp'].dt.minute
        data_filtered_with_nan['day_of_week'] = data_filtered_with_nan['timestamp'].dt.dayofweek
        data_filtered_with_nan['month'] = data_filtered_with_nan['timestamp'].dt.month

        # Separate the data into known and missing parts
        known_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].notna()]
        missing_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].isna()]

        # Define features and target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X_known = known_data[features]
        y_known = known_data['temp_value_imp']
        X_missing = missing_data[features]

        # Train XGBoost regressor
        xgb_model = XGBRegressor(n_estimators=150, learning_rate=0.0575, max_depth=3, random_state=42)
        xgb_model.fit(X_known, y_known)


        missing_data['temp_value_imp'] = xgb_model.predict(X_missing)

        # Combine the imputed data with the known data
        imputed_data = data_filtered_with_nan.copy()
        imputed_data.loc[missing_data.index, 'temp_value_imp'] = missing_data['temp_value_imp']
        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']

        # Plot original vs imputed data for visualization using Plotly Express
        fig = px.line(
            missing_data,
            x=missing_data.index,
            y='temp_value_imp',
            width=1000, height=800,
            title=f'Original vs Imputed Data - {file_name}',
            labels={'temp_value_imp': 'Temperature', 'index': 'Time'},
            color_discrete_sequence=['blue']
        )
        fig.add_scatter(x=original_values.index, y=original_values.values, mode='lines', name='Original', line=dict(color='blue', dash='solid'))
        fig.add_scatter(x=missing_data.index, y=missing_data['temp_value_imp'], mode='lines', name='Imputed', line=dict(color='red', dash='solid'))
        fig.update_traces(name='Imputed', selector=dict(mode='lines'))

        plot_file_name = f"{file_name.split('.')[0]}_{gap_size}.png"
        plot_file_path = os.path.join(output_folder_path, plot_file_name)
        # fig.write_image(plot_file_path, scale=2)


        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']
        imputed_values = missing_data['temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae = mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'File Name': file_name, 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

# Save comparison results to a CSV file
comparison_df = pd.DataFrame(comparison_results)
comparison_df["Year"] = comparison_df["File Name"].str.extract(r"_(\d{4})\.csv")
comparison_df["Suffix"] = comparison_df["File Name"].str.extract(r"e(\d{2})_")
comparison_df = comparison_df.sort_values(by=["Year", "Suffix"])
comparison_df = comparison_df.drop(columns=["Year", "Suffix"])
# comparison_results_file = os.path.join(output_folder_path, 'imputation_comparison_results_450.csv')
# comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nImputation completed for all files. Results saved to {comparison_results_file}.")


**Gap with 600 Values**

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import root_mean_squared_error, mean_absolute_percentage_error,mean_absolute_error
import plotly.express as px
from plotly.io import write_image

input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Plots'
# output_folder_path = '/content/'
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)

        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-06-01') & (data['timestamp'] <= f'{year}-08-30')].reset_index(drop=True)

        # Introduce a gap of 600 consecutive NaN values in 'temp_value_imp'
        start_gap = 6700
        gap_size = 300
        data_filtered_with_nan = data_filtered.copy()
        data_filtered_with_nan.iloc[start_gap:start_gap + gap_size, data_filtered.columns.get_loc('temp_value_imp')] = np.nan

        # Feature Engineering: Add time-based features
        data_filtered_with_nan['time_step'] = range(len(data_filtered_with_nan))
        data_filtered_with_nan['hour'] = data_filtered_with_nan['timestamp'].dt.hour
        data_filtered_with_nan['minute'] = data_filtered_with_nan['timestamp'].dt.minute
        data_filtered_with_nan['day_of_week'] = data_filtered_with_nan['timestamp'].dt.dayofweek
        data_filtered_with_nan['month'] = data_filtered_with_nan['timestamp'].dt.month

        # Separate the data into known and missing parts
        known_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].notna()]
        missing_data = data_filtered_with_nan[data_filtered_with_nan['temp_value_imp'].isna()]

        # Define features and target
        features = ['time_step','hour', 'minute', 'day_of_week', 'month']
        X_known = known_data[features]
        y_known = known_data['temp_value_imp']
        X_missing = missing_data[features]

        # Train XGBoost regressor
        #{'subsample': 0.6, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.0575, 'colsample_bytree': 0.6}
        xgb_model = XGBRegressor(subsample=0.6, n_estimators=150, max_depth=3, learning_rate=0.0575, colsample_bytree= 0.6, random_state=42)
        # xgb_model = XGBRegressor(n_estimators=150, learning_rate=0.0575, max_depth=3, random_state=42)
        xgb_model.fit(X_known, y_known)

        # feature_importances = pd.DataFrame({
        #                     'Feature': features,
        #                     'Importance': xgb_model.feature_importances_
        #                 }).sort_values(by='Importance', ascending=False)

        # print(f"Feature Importance for {file_name}:\n", feature_importances)

        # Predict missing values
        missing_data['temp_value_imp'] = xgb_model.predict(X_missing)


        imputed_data = data_filtered_with_nan.copy()
        imputed_data.loc[missing_data.index, 'temp_value_imp'] = missing_data['temp_value_imp']
        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']

        # Plot original vs imputed data for visualization using Plotly Express
        fig = px.line(
            missing_data,
            x=missing_data.index,
            y='temp_value_imp',
            width=1000, height=800,
            title=f'Original vs Imputed Data - {file_name}',
            labels={'temp_value_imp': 'Temperature', 'index': 'Time'},
            color_discrete_sequence=['blue']
        )
        fig.add_scatter(x=original_values.index, y=original_values.values, mode='lines', name='Original', line=dict(color='blue', dash='solid'))
        fig.add_scatter(x=missing_data.index, y=missing_data['temp_value_imp'], mode='lines', name='Imputed', line=dict(color='red', dash='solid'))
        fig.update_traces(name='Imputed', selector=dict(mode='lines'))

        plot_file_name = f"{file_name.split('.')[0]}_{gap_size}.png"
        plot_file_path = os.path.join(output_folder_path, plot_file_name)
        # fig.write_image(plot_file_path, scale=2)
        # fig.show()

        original_values = data_filtered.loc[missing_data.index, 'temp_value_imp']
        imputed_values = missing_data['temp_value_imp']

        rmse = root_mean_squared_error(original_values, imputed_values)
        mape = mean_absolute_percentage_error(original_values, imputed_values)
        mae = mean_absolute_error(original_values, imputed_values)
        comparison_results.append({'File Name': file_name, 'RMSE': rmse, 'MAPE': mape, 'MAE': mae})
        # print(f"Processed file: {file_name} | RMSE: {rmse:.4f} | MAPE: {mape:.4f}")

# Save comparison results to a CSV file
comparison_df = pd.DataFrame(comparison_results)
comparison_df["Year"] = comparison_df["File Name"].str.extract(r"_(\d{4})\.csv")
comparison_df["Suffix"] = comparison_df["File Name"].str.extract(r"e(\d{2})_")
comparison_df = comparison_df.sort_values(by=["Year", "Suffix"])
comparison_df = comparison_df.drop(columns=["Year", "Suffix"])
# comparison_results_file = os.path.join(output_folder_path, 'imputation_comparison_results_{gap_size}.csv')
# comparison_df.to_csv(comparison_results_file, index=False)

print(f"\nImputation completed for all files. Results saved to {comparison_results_file}.")


In [ ]:
comparison_df

,File Name,RMSE,MAPE,MAE
0,c01m045e01_2021.csv,1.786940,0.065367,1.507172
1,c05m124e01_2021.csv,1.346937,0.044595,1.082356
8,c05m105e08_2021.csv,1.318189,0.044253,1.048672
5,c01m045e01_2022.csv,1.558206,0.049217,1.312973
7,c05m124e01_2022.csv,1.922354,0.058601,1.603477
2,c05m105e08_2022.csv,1.139016,0.036404,0.961973
4,c05m124e01_2023.csv,2.150134,0.058400,1.675614
6,c01m045e01_2023.csv,6.630257,0.196111,5.762467
3,c05m105e08_2023.csv,5.597830,0.154308,4.599021


In [ ]:
fig = px.line(
    missing_data,
    x=missing_data.index,
    y='temp_value_imp',
    width=1000, height=800,
    title=f'Original vs Imputed Data - {file_name}',
    labels={'temp_value_imp': 'Temperature', 'index': 'Time'},
    color_discrete_sequence=['blue']
)

# Add a trace for the original data with a custom legend name
fig.add_scatter(x=original_values.index, y=original_values.values, mode='lines', name='Original', line=dict(color='blue', dash='solid'))

# Add the name for imputed data in the legend (if it isn't appearing yet)
fig.add_scatter(x=missing_data.index, y=missing_data['temp_value_imp'], mode='lines', name='Imputed', line=dict(color='red', dash='solid'))


# Random Search - Hyperparameter Tuning

Hyperparameter Tuning - Limited Selected Dataset

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/Data Imputation_KV/Imputed Data/"

param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': np.linspace(0.01, 0.2, 5),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5)
}

results_dict = {}
file_patterns = ['c01m045e01', 'c05m124e01', 'c05m105e08']
os.makedirs(output_folder_path, exist_ok=True)
comparison_results = []

for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        print(f"\nProcessing file: {file_name}")
        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'  # Default year

        # Read the file
        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value_imp']]


        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-01-01') & (data['timestamp'] <= f'{year}-12-30')].reset_index(drop=True)

        data = pd.read_csv(file_path)
        # Feature Engineering: Add time-based features
        data_filtered['time_step'] = range(len(data_filtered))
        data_filtered['hour'] = data_filtered['timestamp'].dt.hour
        data_filtered['minute'] = data_filtered['timestamp'].dt.minute
        data_filtered['day_of_week'] = data_filtered['timestamp'].dt.dayofweek
        data_filtered['month'] = data_filtered['timestamp'].dt.month

        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X = data_filtered[features]
        y = data_filtered['temp_value_imp']

        X = X.loc[y.notna()]
        y = y.loc[y.notna()]
        xgb_model = XGBRegressor(random_state=42)

        # Perform RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb_model,
            param_distributions=param_dist,
            scoring='neg_root_mean_squared_error',
            n_iter=50,
            cv=5,
            verbose=1,
            random_state=42
        )

        random_search.fit(X, y)

        # Save Best Parameters and RMSE for this file
        best_params = random_search.best_params_
        best_rmse = np.sqrt(-random_search.best_score_)

        results_dict[file_name] = {
            "Best Parameters": best_params,
            "Best RMSE": best_rmse
        }

        print(f"Best RMSE for {file_name}: {best_rmse}")
        print(f"Best Parameters: {best_params}")

results_df = pd.DataFrame.from_dict(results_dict, orient='index')
results_output_path = os.path.join(folder_path, "xgboost_tuning_results_all_data.csv")
results_df.to_csv(results_output_path)

print("\n Hyperparameter tuning completed for all files!")
print(f"Results saved to: {results_output_path}")



Processing file: c01m045e01_2021.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c01m045e01_2021.csv: 2.237122792687971
Best Parameters: {'subsample': 0.6, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.0575, 'colsample_bytree': 0.6}

Processing file: c05m124e01_2021.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m124e01_2021.csv: 2.1015662056553825
Best Parameters: {'subsample': 0.9, 'n_estimators': 150, 'max_depth': 3, 'learning_rate': 0.105, 'colsample_bytree': 0.7}

Processing file: c05m105e08_2022.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m105e08_2022.csv: 2.166334558007655
Best Parameters: {'subsample': 0.7, 'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.0575, 'colsample_bytree': 0.6}

Processing file: c05m105e08_2023.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m105e08_2023.csv: 2.3122788237935734
Best Parameters: {'

In [ ]:
results_df

,Best Parameters,Best RMSE
c01m045e01_2021.csv,"{'subsample': 0.6, 'n_estimators': 150, 'max_d...",2.237123
c05m124e01_2021.csv,"{'subsample': 0.9, 'n_estimators': 150, 'max_d...",2.101566
c05m105e08_2022.csv,"{'subsample': 0.7, 'n_estimators': 50, 'max_de...",2.166335
c05m105e08_2023.csv,"{'subsample': 0.6, 'n_estimators': 100, 'max_d...",2.312279
c05m124e01_2023.csv,"{'subsample': 0.9, 'n_estimators': 50, 'max_de...",2.308694
c01m045e01_2022.csv,"{'subsample': 0.6, 'n_estimators': 100, 'max_d...",2.299720
c01m045e01_2023.csv,"{'subsample': 0.7, 'n_estimators': 50, 'max_de...",2.514818
c05m124e01_2022.csv,"{'subsample': 0.6, 'n_estimators': 100, 'max_d...",2.104847
c05m105e08_2021.csv,"{'subsample': 0.6, 'n_estimators': 150, 'max_d...",2.161010


XGBoost - 2021 - Parameters selection

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# Folder containing the CSV files
input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Resampled/"

# Hyperparameter distribution for Randomized Search
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': np.linspace(0.01, 0.2, 5),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5)
}

# Dictionary to store results
results_dict = {}

file_patterns= ['c05m105e08_2021.csv','c05m105e09_2021.csv','c05m041e09_2021.csv','c05m031e04_2021.csv','c05m124e01_2021.csv','c02m026e02_2021.csv',
                'c01m045e01_2021.csv','c04m122e01_2021.csv','c01m080e01_2021.csv','c03m093e04_2021.csv', 'c05m050e01_2021.csv','c03m102e01_2021.csv',
                'c01m083e01_2021.csv','c05m128e02_2021.csv','c05m040e13_2021.csv','c03m070e01_2021.csv']

os.makedirs(output_folder_path, exist_ok=True)
for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        print(f"\nProcessing file: {file_name}")
        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'  # Default year

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]


        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-01-01') & (data['timestamp'] <= f'{year}-12-30')].reset_index(drop=True)

        # Feature Engineering: Add time-based features
        data_filtered['time_step'] = range(len(data_filtered))
        data_filtered['hour'] = data_filtered['timestamp'].dt.hour
        data_filtered['minute'] = data_filtered['timestamp'].dt.minute
        data_filtered['day_of_week'] = data_filtered['timestamp'].dt.dayofweek
        data_filtered['month'] = data_filtered['timestamp'].dt.month

        # Define Features and Target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X = data_filtered[features]
        y = data_filtered['temp_value']

        # Remove missing target values
        X = X.loc[y.notna()]
        y = y.loc[y.notna()]

        xgb_model = XGBRegressor(random_state=42)

        # Perform RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb_model,
            param_distributions=param_dist,
            scoring='neg_root_mean_squared_error',
            n_iter=50,
            cv=5,
            verbose=1,
            random_state=42
        )

        random_search.fit(X, y)

        # Save Best Parameters and RMSE for this file
        best_params = random_search.best_params_
        best_rmse = np.sqrt(-random_search.best_score_)

        results_dict[file_name] = {
            "Best Parameters": best_params,
            "Best RMSE": best_rmse
        }

        print(f"Best RMSE for {file_name}: {best_rmse}")
        print(f"Best Parameters: {best_params}")

# Convert results dictionary to DataFrame for better readability
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

# Save results to CSV file
results_output_path = os.path.join(input_folder_path, "xgboost_tuning_results_2021.csv")
results_df.to_csv(results_output_path)

print("\n Hyperparameter tuning completed for all files!")
print(f"Results saved to: {results_output_path}")



 Hyperparameter tuning completed for all files!
Results saved to: /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Resampled/xgboost_tuning_results_2022.csv


In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import ast


input_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Resampled/'
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Interpolated XGBoost/'
parameter_file = '/content/xgboost_tuning_results_2021.csv'
os.makedirs(output_folder_path, exist_ok=True)

# Load hyperparameters
xgb_params_df = pd.read_csv(parameter_file)
xgb_params_df.rename(columns={'Unnamed: 0': 'station_id'}, inplace=True)

xgb_params_df['station_id'] = xgb_params_df['station_id'].astype(str).str.lower()
if 'Best Parameters' in xgb_params_df.columns:
    xgb_params_df['Best Parameters'] = xgb_params_df['Best Parameters'].apply(ast.literal_eval)

for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        station_id = file_name.lower()

        # Ensure station_id exists in the hyperparameters file
        params_row = xgb_params_df[xgb_params_df['station_id'] == station_id]
        if params_row.empty:
            print(f"Skipping {file_name}: No matching hyperparameters found.")
            continue

        params = params_row.iloc[0]['Best Parameters']

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]

        # Feature Engineering
        data['time_step'] = range(len(data))
        data['hour'] = data['timestamp'].dt.hour
        data['minute'] = data['timestamp'].dt.minute
        data['day_of_week'] = data['timestamp'].dt.dayofweek
        data['month'] = data['timestamp'].dt.month

        # Train XGBoost
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        known_data = data.dropna()
        X_known = known_data[features]
        y_known = known_data['temp_value']

        X_train, X_test, y_train, y_test = train_test_split(X_known, y_known, test_size=0.2, random_state=42)

        xgb_model = XGBRegressor(
            n_estimators=int(params.get('n_estimators', 100)),
            learning_rate=float(params.get('learning_rate', 0.1)),
            max_depth=int(params.get('max_depth', 5)),
            random_state=42
        )
        xgb_model.fit(X_train, y_train)

        # Evaluate model performance on the test set
        y_pred = xgb_model.predict(X_test)
        mse = root_mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Performance for {file_name} - MSE: {mse:.4f}, R2 Score: {r2:.4f}")

        # Predict missing values
        missing_data = data[data['temp_value'].isna()]
        if not missing_data.empty:
            X_missing = missing_data[features]
            data.loc[data['temp_value'].isna(), 'temp_value'] = xgb_model.predict(X_missing)

        # Saving imputed dataset
        output_path = os.path.join(output_folder_path, file_name)
        data[['timestamp', 'temp_value']].to_csv(output_path, index=False)
        print(f'Processed {file_name} and saved to {output_path}')


Performance for c05m105e08_2021.csv - MSE: 2.2386, R2 Score: 0.8809
Processed c05m105e08_2021.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Interpolated XGBoost/c05m105e08_2021.csv
Performance for c05m105e09_2021.csv - MSE: 1.9181, R2 Score: 0.9224
Processed c05m105e09_2021.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Interpolated XGBoost/c05m105e09_2021.csv
Performance for c05m041e09_2021.csv - MSE: 1.0965, R2 Score: 0.9561
Processed c05m041e09_2021.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2021/Interpolated XGBoost/c05m041e09_2021.csv
Skipping c05m028e04_2021.csv: No matching hyperparameters found.
Skipping c02m051e03_2021.csv: No matching hyperparameters found.
Skipping c01m141e01_2021.csv: No matching hyperparameters found.
Performance for c05m031e04_2021.csv - MSE: 1.4435, R2 Score: 0.9446
Processed c05m031e04_2021.csv and saved to /content/drive/MyDrive/Colab Noteb

XGBoost - 2022 - Parameters selection

In [ ]:
#2022
from sklearn.model_selection import RandomizedSearchCV
# Folder containing the CSV files
input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Resampled/"

# Hyperparameter distribution for Randomized Search
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': np.linspace(0.01, 0.2, 5),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5)
}
# Dictionary to store results
results_dict = {}

file_patterns=['c05m105e08_2022.csv','c05m124e01_2022.csv','c05m041e09_2022.csv','c01m083e01_2022.csv','c01m141e01_2022.csv','c05m028e04_2022.csv',
               'c05m105e09_2022.csv','c02m042e02_2022.csv','c05m050e01_2022.csv','c01m080e01_2022.csv','c03m121e01_2022.csv','c03m093e04_2022.csv',
               'c03m096e01_2022.csv','c01m045e01_2022.csv','c03m102e01_2022.csv','c02m051e03_2022.csv']

os.makedirs(output_folder_path, exist_ok=True)
for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        print(f"\nProcessing file: {file_name}")
        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'  # Default year

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]

        # Adjust the year in the timestamp
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        # Filter the data for the period between June and August
        data_filtered = data[(data['timestamp'] >= f'{year}-01-01') & (data['timestamp'] <= f'{year}-12-30')].reset_index(drop=True)

        # Feature Engineering: Add time-based features
        data_filtered['time_step'] = range(len(data_filtered))
        data_filtered['hour'] = data_filtered['timestamp'].dt.hour
        data_filtered['minute'] = data_filtered['timestamp'].dt.minute
        data_filtered['day_of_week'] = data_filtered['timestamp'].dt.dayofweek
        data_filtered['month'] = data_filtered['timestamp'].dt.month

        # Define Features and Target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X = data_filtered[features]
        y = data_filtered['temp_value']

        # Remove missing target values
        X = X.loc[y.notna()]
        y = y.loc[y.notna()]

        # Initialize XGBoost model
        xgb_model = XGBRegressor(random_state=42)

        # Perform RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb_model,
            param_distributions=param_dist,
            scoring='neg_root_mean_squared_error',
            n_iter=50,
            cv=5,
            verbose=1,
            random_state=42
        )

        random_search.fit(X, y)

        # Save Best Parameters and RMSE for this file
        best_params = random_search.best_params_
        best_rmse = np.sqrt(-random_search.best_score_)

        results_dict[file_name] = {
            "Best Parameters": best_params,
            "Best RMSE": best_rmse
        }

        print(f"Best RMSE for {file_name}: {best_rmse}")
        print(f"Best Parameters: {best_params}")

# Convert results dictionary to DataFrame for better readability
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

# Save results to CSV file
results_output_path = os.path.join(input_folder_path, "xgboost_tuning_results_2022.csv")
results_df.to_csv(results_output_path)

print("\n Hyperparameter tuning completed for all files!")
print(f"Results saved to: {results_output_path}")



Processing file: c05m105e08_2022.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m105e08_2022.csv: 2.1544369683672735
Best Parameters: {'subsample': 0.7, 'n_estimators': 50, 'max_depth': 5, 'learning_rate': 0.0575, 'colsample_bytree': 0.6}

Processing file: c05m124e01_2022.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m124e01_2022.csv: 2.107354232793785
Best Parameters: {'subsample': 0.6, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.0575, 'colsample_bytree': 0.7}

Processing file: c05m041e09_2022.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c05m041e09_2022.csv: 1.7575222318545525
Best Parameters: {'subsample': 0.6, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.0575, 'colsample_bytree': 0.7}

Processing file: c01m083e01_2022.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c01m083e01_2022.csv: 2.3765803132576098
Best Parameters: 

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import ast


input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Resampled/"
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Interpolated XGBoost/'
parameter_file = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Resampled//xgboost_tuning_results_2022.csv'
os.makedirs(output_folder_path, exist_ok=True)

# Load hyperparameters
xgb_params_df = pd.read_csv(parameter_file)
xgb_params_df.rename(columns={'Unnamed: 0': 'station_id'}, inplace=True)

xgb_params_df['station_id'] = xgb_params_df['station_id'].astype(str).str.lower()
if 'Best Parameters' in xgb_params_df.columns:
    xgb_params_df['Best Parameters'] = xgb_params_df['Best Parameters'].apply(ast.literal_eval)

for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        station_id = file_name.lower()

        # Ensure station_id exists in the hyperparameters file
        params_row = xgb_params_df[xgb_params_df['station_id'] == station_id]
        if params_row.empty:
            print(f"Skipping {file_name}: No matching hyperparameters found.")
            continue

        params = params_row.iloc[0]['Best Parameters']

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]

        # Feature Engineering
        data['time_step'] = range(len(data))
        data['hour'] = data['timestamp'].dt.hour
        data['minute'] = data['timestamp'].dt.minute
        data['day_of_week'] = data['timestamp'].dt.dayofweek
        data['month'] = data['timestamp'].dt.month

        # Train XGBoost
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        known_data = data.dropna()
        X_known = known_data[features]
        y_known = known_data['temp_value']

        X_train, X_test, y_train, y_test = train_test_split(X_known, y_known, test_size=0.2, random_state=42)

        xgb_model = XGBRegressor(
            n_estimators=int(params.get('n_estimators', 100)),
            learning_rate=float(params.get('learning_rate', 0.1)),
            max_depth=int(params.get('max_depth', 5)),
            random_state=42
        )
        xgb_model.fit(X_train, y_train)

        # Evaluate model performance on the test set
        y_pred = xgb_model.predict(X_test)
        mse = root_mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Performance for {file_name} - MSE: {mse:.4f}, R2 Score: {r2:.4f}")

        # Predict missing values
        missing_data = data[data['temp_value'].isna()]
        if not missing_data.empty:
            X_missing = missing_data[features]
            data.loc[data['temp_value'].isna(), 'temp_value'] = xgb_model.predict(X_missing)

        # Saving imputed dataset
        output_path = os.path.join(output_folder_path, file_name)
        data[['timestamp', 'temp_value']].to_csv(output_path, index=False)
        print(f'Processed {file_name} and saved to {output_path}')


Performance for c05m105e08_2022.csv - MSE: 2.1274, R2 Score: 0.9096
Processed c05m105e08_2022.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Interpolated XGBoost/c05m105e08_2022.csv
Skipping c03m070e01_2022.csv: No matching hyperparameters found.
Performance for c05m124e01_2022.csv - MSE: 2.2316, R2 Score: 0.9070
Processed c05m124e01_2022.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Interpolated XGBoost/c05m124e01_2022.csv
Skipping c05m040e13_2022.csv: No matching hyperparameters found.
Performance for c05m041e09_2022.csv - MSE: 1.4039, R2 Score: 0.9414
Processed c05m041e09_2022.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2022/Interpolated XGBoost/c05m041e09_2022.csv
Skipping c01m061e01_2022.csv: No matching hyperparameters found.
Performance for c01m083e01_2022.csv - MSE: 3.0667, R2 Score: 0.8568
Processed c01m083e01_2022.csv and saved to /content/drive/MyDrive/Colab Noteb

XGBoost - 2023 - Parameters selection

In [ ]:
#2023
from sklearn.model_selection import RandomizedSearchCV
# Folder containing the CSV files
input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Resampled/"

# Hyperparameter distribution for Randomized Search
param_dist = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7, 10],
    'learning_rate': np.linspace(0.01, 0.2, 5),
    'subsample': np.linspace(0.6, 1.0, 5),
    'colsample_bytree': np.linspace(0.6, 1.0, 5)
}
# Dictionary to store results
results_dict = {}

file_patterns= ['c01m045e01_2023.csv','c01m127e01_2023.csv','c01m061e01_2023.csv','c02m042e02_2023.csv','c01m141e01_2023.csv','c03m070e01_2023.csv',
                'c02m051e03_2023.csv','c03m096e01_2023.csv','c04m122e01_2023.csv','c03m102e01_2023.csv','c03m093e04_2023.csv','c05m105e09_2023.csv',
                'c05m105e08_2023.csv','c05m041e09_2023.csv','c05m040e13_2023.csv','c05m050e01_2023.csv','c05m124e01_2023.csv','c05m128e02_2023.csv']

# os.makedirs(output_folder_path, exist_ok=True)
for file_name in os.listdir(input_folder_path):
    if any(pattern in file_name for pattern in file_patterns) and file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        print(f"\nProcessing file: {file_name}")
        # Determine the year based on the file name
        if '_2022' in file_name:
            year = '2022'
        elif '_2023' in file_name:
            year = '2023'
        else:
            year = '2021'  # Default year

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]

        # Adjust the year in the timestamp
        data['timestamp'] = data['timestamp'].apply(lambda x: x.replace(year=int(year)))
        data_filtered = data[(data['timestamp'] >= f'{year}-01-01') & (data['timestamp'] <= f'{year}-12-30')].reset_index(drop=True)

        # Feature Engineering: Add time-based features
        data_filtered['time_step'] = range(len(data_filtered))
        data_filtered['hour'] = data_filtered['timestamp'].dt.hour
        data_filtered['minute'] = data_filtered['timestamp'].dt.minute
        data_filtered['day_of_week'] = data_filtered['timestamp'].dt.dayofweek
        data_filtered['month'] = data_filtered['timestamp'].dt.month

        # Define Features and Target
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        X = data_filtered[features]
        y = data_filtered['temp_value']

        # Remove missing target values
        X = X.loc[y.notna()]
        y = y.loc[y.notna()]

        # Initialize XGBoost model
        xgb_model = XGBRegressor(random_state=42)

        # Perform RandomizedSearchCV
        random_search = RandomizedSearchCV(
            estimator=xgb_model,
            param_distributions=param_dist,
            scoring='neg_root_mean_squared_error',
            n_iter=50,
            cv=5,
            verbose=1,
            random_state=42
        )

        random_search.fit(X, y)

        # Save Best Parameters and RMSE for this file
        best_params = random_search.best_params_
        best_rmse = np.sqrt(-random_search.best_score_)

        results_dict[file_name] = {
            "Best Parameters": best_params,
            "Best RMSE": best_rmse
        }

        print(f"Best RMSE for {file_name}: {best_rmse}")
        print(f"Best Parameters: {best_params}")

# Convert results dictionary to DataFrame for better readability
results_df = pd.DataFrame.from_dict(results_dict, orient='index')

# Save results to CSV file
results_output_path = os.path.join(input_folder_path, "xgboost_tuning_results_2023.csv")
results_df.to_csv(results_output_path)

print("\n Hyperparameter tuning completed for all files!")
print(f"Results saved to: {results_output_path}")



Processing file: c01m045e01_2023.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c01m045e01_2023.csv: 2.494295181818207
Best Parameters: {'subsample': 0.8, 'n_estimators': 100, 'max_depth': 10, 'learning_rate': 0.0575, 'colsample_bytree': 0.8}

Processing file: c01m127e01_2023.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c01m127e01_2023.csv: 2.428973456563194
Best Parameters: {'subsample': 1.0, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.0575, 'colsample_bytree': 1.0}

Processing file: c01m061e01_2023.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c01m061e01_2023.csv: 2.567069071540638
Best Parameters: {'subsample': 0.7, 'n_estimators': 100, 'max_depth': 5, 'learning_rate': 0.0575, 'colsample_bytree': 1.0}

Processing file: c02m042e02_2023.csv
Fitting 5 folds for each of 50 candidates, totalling 250 fits
Best RMSE for c02m042e02_2023.csv: 2.266491587883265
Best Parameters: {

In [ ]:
import os
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, root_mean_squared_error
import ast

input_folder_path = "/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Resampled/"
output_folder_path = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Interpolated XGBoost/'
parameter_file = '/content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Resampled/xgboost_tuning_results_2023.csv'
os.makedirs(output_folder_path, exist_ok=True)

# Load hyperparameters
xgb_params_df = pd.read_csv(parameter_file)
xgb_params_df.rename(columns={'Unnamed: 0': 'station_id'}, inplace=True)

xgb_params_df['station_id'] = xgb_params_df['station_id'].astype(str).str.lower()
if 'Best Parameters' in xgb_params_df.columns:
    xgb_params_df['Best Parameters'] = xgb_params_df['Best Parameters'].apply(ast.literal_eval)

for file_name in os.listdir(input_folder_path):
    if file_name.endswith('.csv'):
        file_path = os.path.join(input_folder_path, file_name)
        station_id = file_name.lower()

        # Ensure station_id exists in the hyperparameters file
        params_row = xgb_params_df[xgb_params_df['station_id'] == station_id]
        if params_row.empty:
            print(f"Skipping {file_name}: No matching hyperparameters found.")
            continue

        params = params_row.iloc[0]['Best Parameters']

        data = pd.read_csv(file_path)
        data['timestamp'] = pd.to_datetime(data['timestamp'])
        data = data[['timestamp', 'temp_value']]

        # Feature Engineering
        data['time_step'] = range(len(data))
        data['hour'] = data['timestamp'].dt.hour
        data['minute'] = data['timestamp'].dt.minute
        data['day_of_week'] = data['timestamp'].dt.dayofweek
        data['month'] = data['timestamp'].dt.month

        # Train XGBoost
        features = ['time_step', 'hour', 'minute', 'day_of_week', 'month']
        known_data = data.dropna()
        X_known = known_data[features]
        y_known = known_data['temp_value']

        X_train, X_test, y_train, y_test = train_test_split(X_known, y_known, test_size=0.2, random_state=42)

        xgb_model = XGBRegressor(
            n_estimators=int(params.get('n_estimators', 100)),
            learning_rate=float(params.get('learning_rate', 0.1)),
            max_depth=int(params.get('max_depth', 5)),
            random_state=42
        )
        xgb_model.fit(X_train, y_train)

        # Evaluate model performance on the test set
        y_pred = xgb_model.predict(X_test)
        mse = root_mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print(f"Performance for {file_name} - MSE: {mse:.4f}, R2 Score: {r2:.4f}")

        # Predict missing values
        missing_data = data[data['temp_value'].isna()]
        if not missing_data.empty:
            X_missing = missing_data[features]
            data.loc[data['temp_value'].isna(), 'temp_value'] = xgb_model.predict(X_missing)

        # Saving imputed dataset
        output_path = os.path.join(output_folder_path, file_name)
        data[['timestamp', 'temp_value']].to_csv(output_path, index=False)
        print(f'Processed {file_name} and saved to {output_path}')


Performance for c01m045e01_2023.csv - MSE: 1.2069, R2 Score: 0.9766
Processed c01m045e01_2023.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Interpolated XGBoost/c01m045e01_2023.csv
Skipping c01m080e01_2023.csv: No matching hyperparameters found.
Performance for c01m127e01_2023.csv - MSE: 2.1016, R2 Score: 0.9328
Processed c01m127e01_2023.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Interpolated XGBoost/c01m127e01_2023.csv
Performance for c01m061e01_2023.csv - MSE: 3.0363, R2 Score: 0.8909
Processed c01m061e01_2023.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Interpolated XGBoost/c01m061e01_2023.csv
Skipping c01m083e01_2023.csv: No matching hyperparameters found.
Performance for c02m042e02_2023.csv - MSE: 2.0941, R2 Score: 0.9124
Processed c02m042e02_2023.csv and saved to /content/drive/MyDrive/Colab Notebooks/Data/Avamet/Temperature/2023/Interpolated XGBoost/c02m042e02

In [ ]:
#files with consecutive missing count < 150
x150= ['c05m105e08_2021.csv','c05m105e09_2021.csv','c05m041e09_2021.csv','c05m028e04_2021.csv','c01m141e01_2021.csv','c05m031e04_2021.csv',
        'c05m124e01_2021.csv','c02m026e02_2021.csv','c01m045e01_2021.csv','c04m122e01_2021.csv','c01m127e01_2021.csv','c01m080e01_2021.csv',
        'c03m093e04_2021.csv','c05m050e01_2021.csv','c03m102e01_2021.csv','c02m042e02_2021.csv','c01m083e01_2021.csv','c05m128e02_2021.csv',
        'c05m040e13_2021.csv','c03m070e01_2021.csv']

#files with consecutive missing count < 50
x50=['c05m105e08_2021.csv','c05m105e09_2021.csv','c05m041e09_2021.csv','c05m031e04_2021.csv','c05m124e01_2021.csv','c02m026e02_2021.csv',
     'c01m045e01_2021.csv','c04m122e01_2021.csv','c01m080e01_2021.csv','c03m093e04_2021.csv', 'c05m050e01_2021.csv','c03m102e01_2021.csv',
     'c01m083e01_2021.csv','c05m128e02_2021.csv','c05m040e13_2021.csv','c03m070e01_2021.csv']

In [ ]:
#files with consecutive missing count < 150
countx=['c05m105e08_2022.csv','c05m124e01_2022.csv','c05m041e09_2022.csv','c01m083e01_2022.csv','c01m141e01_2022.csv','c05m028e04_2022.csv',
        'c05m105e09_2022.csv','c02m042e02_2022.csv','c04m122e01_2022.csv','c05m050e01_2022.csv','c01m080e01_2022.csv','c03m121e01_2022.csv',
        'c03m093e04_2022.csv','c03m096e01_2022.csv','c05m031e04_2022.csv','c01m045e01_2022.csv','c03m102e01_2022.csv','c02m051e03_2022.csv']

#files with consecutive missing count < 50
yy=['c05m105e08_2022.csv','c05m124e01_2022.csv','c05m041e09_2022.csv','c01m083e01_2022.csv','c01m141e01_2022.csv','c05m028e04_2022.csv',
    'c05m105e09_2022.csv','c02m042e02_2022.csv','c05m050e01_2022.csv','c01m080e01_2022.csv','c03m121e01_2022.csv','c03m093e04_2022.csv',
    'c03m096e01_2022.csv','c01m045e01_2022.csv','c03m102e01_2022.csv','c02m051e03_2022.csv']


In [ ]:
#files with consecutive missing count < 150
xyz=['c01m045e01_2023.csv','c01m080e01_2023.csv','c01m127e01_2023.csv','c01m061e01_2023.csv','c01m083e01_2023.csv','c02m042e02_2023.csv',
     'c01m141e01_2023.csv','c02m026e02_2023.csv','c02m119e02_2023.csv','c03m070e01_2023.csv','c02m051e03_2023.csv','c05m028e04_2023.csv',
     'c03m096e01_2023.csv','c04m122e01_2023.csv','c03m102e01_2023.csv','c03m093e04_2023.csv','c05m105e09_2023.csv','c05m105e08_2023.csv',
     'c05m041e09_2023.csv','c05m040e13_2023.csv','c05m050e01_2023.csv','c05m124e01_2023.csv','c05m128e02_2023.csv']

#files with consecutive missing count < 50
abc= ['c01m045e01_2023.csv','c01m127e01_2023.csv','c01m061e01_2023.csv','c02m042e02_2023.csv','c01m141e01_2023.csv','c03m070e01_2023.csv',
      'c02m051e03_2023.csv','c03m096e01_2023.csv','c04m122e01_2023.csv','c03m102e01_2023.csv','c03m093e04_2023.csv','c05m105e09_2023.csv',
      'c05m105e08_2023.csv','c05m041e09_2023.csv','c05m040e13_2023.csv','c05m050e01_2023.csv','c05m124e01_2023.csv','c05m128e02_2023.csv']
